In [1]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap


from scipy.special import binom

In [2]:
log_file_path = "/Users/mparedes/Desktop/mpox_rhino/300_region_updated_skyline.log"


In [3]:
burnin_percent = 0.30

In [4]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        deme = name.split(".")[1]# the syntax here is "Ne.region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[2]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)

In [5]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

In [68]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "SkylineNe" in i:
            deme = i.split(".")[1]
            #print(deme)
            interval = i.split(".")[2]
            #print(interval)
            #print(interval)
            #print(i)
            next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = 10**mean_log
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
            try:
                next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
                diff_series = np.subtract(local_series, next_local_series)
                #print(local_series)
                #print(next_local_series)
                #print(diff_series)
                diff_mean_log = diff_series.mean()
                diff_median_log = np.median(diff_series)
                diff_hpd_95 = az.hdi(diff_series, 0.95)
                diff_lower_hpd_log_95 = diff_hpd_95[0]
                diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
                diff_upper_hpd_log_95 = diff_hpd_95[1]
                diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
                diff_hpd_50 = az.hdi(diff_series, 0.50)
                diff_lower_hpd_log_50 = diff_hpd_50[0]
                diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
                diff_upper_hpd_log_50 = diff_hpd_50[1]
                diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
            except KeyError:
                pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"deme":deme, "interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                   "diff_mean_Ne_log":diff_mean_log, "diff_mean_log":diff_mean_log,
                                                   "diff_upper_hpd_log_95":diff_upper_hpd_log_95,"diff_lower_hpd_log_95":diff_lower_hpd_log_95, 
                                                   "diff_upper_hpd_log_50":diff_upper_hpd_log_50,"diff_lower_hpd_log_50":diff_lower_hpd_log_50,
                                                   "diff_upper_hpd_linear":diff_upper_hpd_linear_95,"diff_lower_hpd_linear":diff_lower_hpd_linear_95,
                                                   "diff_upper_hpd_linear_50":diff_upper_hpd_linear_50,"diff_lower_hpd_linear_50":diff_lower_hpd_linear_50})
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [63]:
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
Ne_df

25137


,sample,SkylineNe.CentralEurope.1,SkylineNe.CentralEurope.2,SkylineNe.CentralEurope.3,SkylineNe.CentralEurope.4,SkylineNe.CentralEurope.5,SkylineNe.CentralEurope.6,SkylineNe.CentralEurope.7,SkylineNe.CentralEurope.8,SkylineNe.CentralEurope.9,...,SkylineNe.WesternEurope.2,SkylineNe.WesternEurope.3,SkylineNe.WesternEurope.4,SkylineNe.WesternEurope.5,SkylineNe.WesternEurope.6,SkylineNe.WesternEurope.7,SkylineNe.WesternEurope.8,SkylineNe.WesternEurope.9,SkylineNe.WesternEurope.10,SkylineNe.WesternEurope.11\n
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n
1,1000,0.10907159157724362,0.937061001446744,-0.36822676743696764,0.6377191083682311,1.0649902980361703,-0.7607644850909145,-0.3987028862929026,-0.7476790181417562,0.48437130213016955,...,-0.08538475753913224,-0.8604302803386132,-0.7267490367167464,0.21426592701034136,-1.4300836064384477,0.8653250483369149,0.39531741062634984,-1.5526957414545677,0.6030287775584333,-0.62589961302214\n
2,2000,1.0184564391502704,0.7540381061278125,0.11840759885786006,0.49526177696195206,1.1969547510530734,-0.6175860379966205,-0.2639750207140214,-0.7275862635058624,-0.017718470351097623,...,0.24069526789420792,-1.3014392977864984,-1.1238443625682988,1.1548478806046105,-1.3784150533926094,0.7241877776179761,0.911762205261385,-1.8672840084530422,0.6946886445453182,-0.02676348141115878\n
3,3000,0.3145432367167688,-0.8046656860279442,1.5309334652715079,3.9446399297014714,0.4199057559932893,-0.01908664564588672,-0.9544962494029606,0.08805929769980143,1.1108014774970065,...,0.2727901541601254,1.3832577962360686,0.41778360282972316,-0.27920233009606765,-0.42358014146657685,-1.406898610575779,-0.26671442089351516,-0.7662644953854858,-0.34854576899716877,0.2434019795640711\n
4,4000,0.2958372227754861,-0.21682221080859942,2.2093132333279493,4.10415274873988,0.5186024467387257,0.46493896281494207,-2.0013385189631,-0.7433541754168079,0.9439272101991396,...,0.1054481623267984,0.9221111248320366,0.3211458050074005,0.12311225222707546,-0.20312323725309828,-1.6539449511155255,-0.09310765336835414,-0.5574430577535477,0.17311148106796959,0.8676952348136744\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25132,25132000,-1.8061006074425157,-2.8296481886894527,-3.0123583372156744,-3.241937510893199,-2.659602779987661,-3.7145828509240975,-4.590028797302542,-3.0328346336813365,-3.728113412368398,...,-1.019275354871899,-0.13864259249008393,-1.6202891039977114,-1.4257831823302942,-0.7633244039794115,-1.6158145789221359,-3.1390977723408633,-4.122285733524883,-5.095436186250304,-6.066684393610581\n
25133,25133000,-1.7816581160252074,-2.680554040104338,-2.9435255488361816,-3.5133508136810803,-2.9324808182830777,-3.751592780295514,-4.581785695832833,-2.635725918409052,-3.6254459391581912,...,-0.6674346473778661,0.11950322204488599,-1.4384055569736778,-1.175000597174307,-0.5013674194520824,-1.192523509387394,-2.7474764931918627,-3.681337321041181,-4.763772884230624,-5.592252120556306\n
25134,25134000,-1.7756476832484733,-2.73526063457005,-2.7949657488634956,-3.118551744840723,-2.709349832091477,-3.7187984607553886,-4.339678194358086,-2.409296481574977,-3.788371463602281,...,-0.5057827675794345,0.4418714274944951,-1.1504475459304513,-1.0329804088897183,-0.6819461784421056,-1.4195922619066097,-2.637350991247762,-3.593756172949198,-4.695454198830119,-5.915257302752934\n
25135,25135000,-1.7756476832484733,-2.73526063457005,-2.7949657488634956,-3.118551744840723,-2.709349832091477,-3.7187984607553886,-4.339678194358086,-2.409296481574977,-3.788371463602281,...,-0.5057827675794345,0.2980229534960081,-0.4621855529922374,-0.6319162281209743,-0.6819461784421056,-1.4195922619066097,-2.637350991247762,-3.593756172949198,-4.695454198830119,-5.915257302752934\n


In [69]:
ne_summary = generate_summary_df(Ne_df)
ne_summary

/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_2929/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_2929/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_2929/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_2929/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var

/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_2929/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_2929/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_2929/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_2929/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var

,deme,interval,mean_Ne_log,mean_Ne_linear,median_Ne_log,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,...,diff_mean_Ne_log,diff_mean_log,diff_upper_hpd_log_95,diff_lower_hpd_log_95,diff_upper_hpd_log_50,diff_lower_hpd_log_50,diff_upper_hpd_linear,diff_lower_hpd_linear,diff_upper_hpd_linear_50,diff_lower_hpd_linear_50
0,CentralEurope,1,-0.619579,0.240116,-0.626283,1.175171,-2.398074,-0.113573,-1.316546,3.238697,...,0.554389,0.554389,2.312816,-1.241196,1.178214,-0.064808,10.102837,0.289038,3.248567,0.937248
0,CentralEurope,2,-1.173968,0.066993,-1.193645,1.143891,-3.458345,-0.500129,-2.089253,3.138957,...,0.518957,0.518957,2.307523,-1.306430,1.142586,-0.089118,10.049501,0.270785,3.134866,0.914737
0,CentralEurope,3,-1.692925,0.020280,-1.716103,0.815805,-4.249056,-0.871514,-2.661960,2.260995,...,0.457370,0.457370,2.292127,-1.489014,1.137794,-0.110356,9.895963,0.225595,3.119877,0.895515
0,CentralEurope,4,-2.150295,0.007075,-2.275087,0.799533,-4.772135,-1.483074,-3.295857,2.224503,...,0.317505,0.317505,2.055258,-1.579524,0.970033,-0.299009,7.808849,0.206073,2.638032,0.741553
0,CentralEurope,5,-2.467800,0.003406,-2.708528,0.782677,-4.988029,-2.133207,-3.714047,2.187320,...,0.834833,0.834833,2.654308,-1.123785,1.444538,0.195153,14.215141,0.325047,4.239891,1.215497
0,CentralEurope,6,-3.302633,0.000498,-3.528065,-0.109776,-5.451912,-3.135618,-4.605809,0.896035,...,0.845807,0.845807,2.747532,-1.033836,1.548150,0.274198,15.604073,0.355640,4.702764,1.315475
0,CentralEurope,7,-4.148440,0.000071,-4.397259,-1.184700,-6.176258,-3.995954,-5.177643,0.305838,...,0.197106,0.197106,2.155706,-1.858931,0.861643,-0.469324,8.633987,0.155839,2.367046,0.625425
0,CentralEurope,8,-4.345547,0.000045,-4.499083,-1.384542,-7.262207,-3.801575,-5.450462,0.250439,...,0.033740,0.033740,1.839237,-1.935446,0.640353,-0.676823,6.291735,0.144360,1.897151,0.508229
0,CentralEurope,9,-4.379287,0.000042,-4.495814,-0.821389,-8.024881,-3.478662,-5.498358,0.439820,...,-0.017091,-0.017091,2.016823,-1.898445,0.701802,-0.635762,7.514416,0.149801,2.017385,0.529532
0,CentralEurope,10,-4.362196,0.000043,-4.459270,0.537221,-8.026046,-3.280985,-5.664017,1.711245,...,-0.057519,-0.057519,1.943604,-2.002090,0.573215,-0.783829,6.983875,0.135053,1.773962,0.456654


In [70]:
ne_summary.lower_hpd_linear_50

0     0.268060
0     0.123780
0     0.069811
0     0.037036
0     0.024379
0     0.009994
0     0.005641
0     0.004294
0     0.004093
0     0.003469
0     0.002708
0     1.161257
0     1.940444
0     1.515519
0     1.128944
0     1.364830
0     1.576468
0     0.553163
0     0.135855
0     0.033303
0     0.021364
0     0.014988
0     0.652051
0     0.862332
0     1.694392
0     1.504245
0     0.410222
0     0.117212
0     0.039550
0     0.022113
0     0.014076
0     0.011289
0     0.008475
0     0.763204
0     1.144764
0     1.917914
0     4.853767
0    18.178494
0    16.588627
0    10.780476
0     0.214932
0     0.081466
0     0.028185
0     0.022723
0     0.398347
0     0.245943
0     0.162470
0     0.135848
0     0.252734
0     0.205096
0     0.128833
0     0.036949
0     0.012847
0     0.008323
0     0.006850
Name: lower_hpd_linear_50, dtype: float64

In [71]:
test = ne_summary
test['days'] = (test.interval.astype(int))*25
test['date'] = dt.strptime("2023-01-03",  "%Y-%m-%d") - test.days.map(timedelta)
#test.date = test.date.astype(str)

In [83]:
domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.5).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
    alt.Y('lower_hpd_linear_50',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_50' ),
    color=alt.Color('deme:N', scale=alt.Scale(domain=domain, range=range_) )
).properties(
    width=850,
    height=400
).transform_filter(
    (datum.upper_hpd_linear < 60) &(datum.upper_hpd_linear_50 < 20)
)

# band = alt.Chart(test).mark_area(
#     opacity=0.3, interpolate='monotone'
# ).encode(
#     alt.X('date:T', axis=alt.Axis(title="", grid=False)),
#     alt.Y('lower_hpd_linear'),#axis=None),#, scale=alt.Scale(domain=(0, 13))),
#     alt.Y2('upper_hpd_linear'),
#     color=alt.Color('deme:N', title = "Region", legend=alt.Legend(orient = "right",offset = -135, labelFontSize = 16, titleFontSize = 16, symbolSize = 110))
# ).properties(
#     width=850,
#     height=400
# ).transform_filter(
#     (datum.upper_hpd_linear < 20) &(datum.upper_hpd_linear_50 < 30)
# )
# band
# ne_plot = line + band
# (ne_plot).configure_axis(
#     labelFontSize=20,
#     titleFontSize=20
# )
line

alt.Chart(...)